In [3]:
import requests
import pandas as pd

# Get HC3 data

In [8]:
API_URL = "https://datasets-server.huggingface.co/parquet?dataset=Hello-SimpleAI/HC3"
def query():
    response = requests.request("GET", API_URL)
    return response.json()
data = query()
print(data['parquet_files'][0])

{'dataset': 'Hello-SimpleAI/HC3', 'config': 'all', 'split': 'train', 'url': 'https://huggingface.co/datasets/Hello-SimpleAI/HC3/resolve/refs%2Fconvert%2Fparquet/all/hc3-train.parquet', 'filename': 'hc3-train.parquet', 'size': 39283466}


In [9]:
df = pd.read_parquet(data['parquet_files'][0]['url'])
df.index = df['id']
df.drop(['id'], axis=1, inplace=True)
df.head()

,question,human_answers,chatgpt_answers,source
id,,,,
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,reddit_eli5
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,reddit_eli5
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,reddit_eli5
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,reddit_eli5
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,reddit_eli5


In [10]:
# create questionid table to assign each question a unique ID
q_ids = df.groupby("question").size().reset_index().drop(labels=0, axis=1).reset_index().rename(columns={"index": "q_id"})
print(f"original HC3 df shape: {df.shape}, unique questions: {q_ids.shape[0]}")
q_ids

original HC3 df shape: (24322, 4), unique questions: 23492


,q_id,question
0,0,A googolplex is precisely as far from infinit...
1,1,"Red sky at night , Sailor 's delight . Red sk..."
2,2,The Great Digg Migration . I 've seen this p...
3,3,knots in your back . What is their cause and...
4,4,""" Demons run when a good man goes to war "" Exp..."
...,...,...
23487,23487,“Correct” answer on Visa credit quiz doesn't m...
23488,23488,“Debt Settlement Order” Text Spams – How do th...
23489,23489,“International credit report” for French natio...
23490,23490,“Occupation” field on IRS Form 1040


In [11]:
# explode human and chatgpt answers
human = df[["question", "human_answers", "source"]].explode(column="human_answers").rename(columns={"human_answers": "answer"})
# assign question ids using merge
human = human.merge(q_ids, on="question", how="inner")
human["true_label"] = "real"
human.dropna(inplace=True)

chatgpt = df[["question", "chatgpt_answers", "source"]].explode(column="chatgpt_answers").rename(columns={"chatgpt_answers": "answer"})
# assign question ids using merge
chatgpt = chatgpt.merge(q_ids, on="question", how="inner")
chatgpt["true_label"] = "fake"
chatgpt.dropna(inplace=True)

# question / answer / true_label (real/fake) / source
clean_df = pd.concat((human,chatgpt)).reset_index().drop(columns="index")[["q_id", "question", "answer", "true_label", "source"]]
print(clean_df.shape)
clean_df.head()


(85449, 5)


,q_id,question,answer,true_label,source
0,19437,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",real,reddit_eli5
1,19437,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",real,reddit_eli5
2,19437,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",real,reddit_eli5
3,6748,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,real,reddit_eli5
4,6748,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",real,reddit_eli5
